<a href="https://colab.research.google.com/github/CameronKenworthyCode/python/blob/main/PLSS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Built T-R-S map data by querying BLM public land survey**

#Setup

In [ ]:
!pip install requests
!pip install bs4
!pip install -U -q PyDrive

import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from google.colab import drive
from oauth2client.client import GoogleCredentials
import random
import json

auth.authenticate_user() #establish connection to a gdrive to write the eventual csv file to
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
gdrive = GoogleDrive(gauth)

local_download_path = os.path.expanduser('~/data')
try:
  os.makedirs(local_download_path)
except: pass

drive.mount('/content/drive/', force_remount=True)

file_list = gdrive.ListFile(
    {'q': "'1DWFETkLt-dWcA7CfymejJe7O-UJxSR4B' in parents and trashed=False"}).GetList() 

EWA_TRS = pd.read_csv('drive/My Drive/NEWICC/PLSS/EWA_TRS.csv', index_col=0)

# TODO

~~- generate URLS~~

~~- batch query~~
~~- save geometry long/lat coords~~

  ~~- number of coords given varies per section~~
    
    - get corners
      - get all points along each edge
    - infer outline of township range from edges of sections
- infer shape from coords
- save all as csv
- code to calculate if coord is in shape

#Functions

In [2]:
# generate set of strings in form WA330##0N0##0E0SN##0 or WA330##0N0##0W0SN##0
# return list of these strings of possible sections 1-60 for each T/R pair
def generate_url(town, _range):
  t_ = town[:-1]
  if int(t_) < 10:
    t_ = '0' + t_
  r_ = _range[:-1]
  if int(r_) < 10:
    r_ = '0' + r_
  r_ = r_ + '0' + _range[-1:]
  base_string = 'WA330'+ t_ + '0N0' + r_ + '0SN'
  section_list = []
  for i in range(1, 60):
    s_ = str(i) + '0'
    if i < 10:
      s_ = '0' + str(i) + '0'
    section_list.append(base_string+s_)
  
  return section_list

#Generate URL Queries

T/R pairs in Washington State:

TOWN: 1N -> 41N

RANGE: 16W -> 47E


In [18]:
# generate urls in batches of 25 queries
trs = []

# for all T/R where R is West
for i in range(1,42):
  for j in range(1,17):
    trs.append([str(i)+'N', str(j)+'W'])

# for all T/R where R is East
for i in range(1, 42):
  for j in range(1, 48):
    trs.append([str(i)+'N', str(j)+'E'])

# All the section url words for each T/R
url_dict = {}
for pair in trs:
  key = pair[0] + '-' + pair[1]
  value = generate_url(pair[0], pair[1])
  url_dict[key] = value

#strings to build url query
base_url = 'https://gis.blm.gov/arcgis/rest/services/Cadastral/BLM_Natl_PLSS_CadNSDI/MapServer/exts/CadastralSpecialServices/GetLatLon?trs='
built_url = ''
base_url_end = '&returnalllevels=&f=pjson'

# go through each T/R pair, and take the next 25 section urls to build a query, save the results and continue to the next 25
count = 0
keys = {} # for building PLSS_dict
urls = [] # for sending batches to requests.get()
for i in range(len(trs)):
  # for each T/R pair
  for j in range(0, 59):
    # for each possible section in each T/R pair
    if count == 25: # url query maxed, store and refresh
      urls.append(base_url+built_url+base_url_end)
      built_url = ''
      count = 0
    count += 1
    built_url = built_url + url_dict[trs[i][0] + '-' + trs[i][1]][j] + '+%7C+'
    # create dict of WA330##0N0##0E0SN##0 keys corresponding to regular T/R/S notation
    # use dict to correctly put away coordinates to correct T/R/S in PLSS_dict
    keys[url_dict[trs[i][0] + '-' + trs[i][1]][j]] = trs[i][0] + '-' + trs[i][1] + '-' + str(j+1) 

# Query gis.blm.gov for T/R/S coordinate shapes

In [ ]:
'''

ALREADY RAN
TAKES A LONG TIME
JUST LOAD CSV INSTEAD

'''
# fill dict with coordinate outlines of each section in every T/R
# by querying gis.blm.gov

PLSS_dict_raw = {}

for i in range(len(urls)):
  url = urls[i]
  parser = html2text.HTML2Text()
  parser.ignore_links = True
  result = requests.get(url)
  src = result.content
  soup = BeautifulSoup(src, 'lxml')
  result = json.loads(soup.text)
  print(str(round(i/len(urls)*100, 3)) + ': ' + str(len(result['features'])))
  for i in range(len(result['features'])):
    #{'TT-RR-SS': [[List of section defining coordinates]]}
    PLSS_dict_raw[keys[result['features'][i]['attributes']['landdescription']]] = result['features'][i]['geometry']['rings'][0]

plss_df = pd.DataFrame({ key:pd.Series(value) for key, value in PLSS_dict_raw.items() })
plss_df = plss_df.T
plss_df.to_csv('plss.csv')
!cp plss.csv "drive/My Drive/NEWICC/PLSS/"

# lookup table for easy search
tr_index = {}

# for each valid T/R/S word
# create a row of their integer T/R/S values
for index in plss_df.index:
  trs_list = index.split('-')
  if trs_list[1][-1:] == 'W':
    continue
  town = int(trs_list[0][:-1])
  range = int(trs_list[1][:-1])
  section = int(trs_list[2])
  tr_index[index] = {"town": town, 'range':range, 'section':section}

# build final dataframe
tr_index_df = pd.DataFrame(tr_index).T
trs_df = tr_index_df.join(plss_df).dropna(axis=1, how='all')
trs_df.to_csv('EWA_trs.csv')
!cp EWA_trs.csv "drive/My Drive/NEWICC/PLSS/"

#Get missing Forest Service T/R data

In [7]:
EWA_TRS

,town,range,section,0,1,2,3,4,5,6,...,484,485,486,487,488,489,490,491,492,493
1N-2E-1,1,2,1,"[-122.49560682884437, 45.60663320323578]","[-122.49555497179796, 45.603431328122525]","[-122.49554947321012, 45.60309191907543]","[-122.49554934205608, 45.603063566285954]","[-122.49553675935388, 45.60035327312725]","[-122.49549161182435, 45.59655384522284]","[-122.50391604220322, 45.59647134392957]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1N-2E-2,1,2,2,"[-122.51625983204816, 45.60554041526393]","[-122.52371076174691, 45.605489628437965]","[-122.52374733306046, 45.60659662841362]","[-122.51633690390629, 45.60669027220311]","[-122.51625983204816, 45.60554041526393]",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1N-3E-1,1,3,1,"[-122.38341989544209, 45.592596774245294]","[-122.38551481160042, 45.59258035832261]","[-122.38556473637065, 45.594965788264815]","[-122.3855830646974, 45.59535819536909]","[-122.38429977370333, 45.59537915944071]","[-122.38338148348055, 45.59538177366347]","[-122.38262119164489, 45.59538394222992]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1N-3E-2,1,3,2,"[-122.39350988500274, 45.60713299809458]","[-122.39358574503353, 45.60350178883791]","[-122.393586019918, 45.60348821871675]","[-122.39359860531515, 45.60288652647622]","[-122.39869714372881, 45.60286409518015]","[-122.40339992575082, 45.60284317161407]","[-122.40478427811267, 45.60283701244676]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1N-3E-3,1,3,3,"[-122.42956799730092, 45.59248095938393]","[-122.43464217520071, 45.59247799112288]","[-122.43464501836858, 45.596082198047]","[-122.43464785884152, 45.59968639222064]","[-122.43465069751781, 45.60329058554351]","[-122.43465353439748, 45.60689477860008]","[-122.42949157312813, 45.60691175027605]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41N-7E-32,41,7,32,"[-121.90557517462155, 48.99956073526347]","[-121.90558517376897, 48.99609573507926]","[-121.90560517296213, 48.9926257353445]","[-121.90644517231294, 48.992635735110035]","[-121.91198517339996, 48.9926357345206]","[-121.91752524545387, 48.9926357339312]","[-121.923025175391, 48.99265580418845]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41N-7E-33,41,7,33,"[-121.88352517077666, 48.99922573820132]","[-121.88356524102824, 48.99606573786009]","[-121.88361516939173, 48.992575737665376]","[-121.88442516950363, 48.992575737665376]","[-121.88910517025, 48.99259573721457]","[-121.88989517027042, 48.99259573662518]","[-121.89460517115405, 48.99260573639674]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41N-7E-34,41,7,34,"[-121.86169516703981, 48.99888574051526]","[-121.86168516609577, 48.99591574029459]","[-121.86167516515171, 48.992305740018146]","[-121.86256516562956, 48.992315739847925]","[-121.86716516601001, 48.99237573937401]","[-121.86802516635062, 48.99238573918977]","[-121.87264516682256, 48.99244573863155]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41N-7E-35,41,7,35,"[-121.83964516319497, 48.99854574286584]","[-121.83967516243386, 48.9959357426287]","[-121.83972516176426, 48.9922357423322]","[-121.84072509088007, 48.9922357423322]","[-121.84618516256805, 48.99225574201789]","[-121.85163516324337, 48.99226566994615]","[-121.85710516490848, 48.992285740352486]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
# a lot of T/Rs are missing from PLSS

range_i = range(EWA_TRS['town'].unique().min(), EWA_TRS['town'].unique().max())
range_j = range(EWA_TRS['range'].unique().min(), EWA_TRS['range'].unique().max())
missing_TR = []
for i in range_i:
  for j in range_j:
    if len(EWA_TRS.query('town == @i and range == @j')) == 0:
      missing_TR.append([i, j])

len(missing_TR) # 329 missing T/R in EWA, checking map they are all Forest Service 

329